In [ ]:
###SASSIE ALTO_ALAMO DATA USE AND VISUALIZATION##
import numpy as np
import xarray as xr
import glob 
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import cartopy
import matplotlib
import os
import sys
import pandas as pd

#local directory where the Profiling Float data is stored
dir_in = '/Users/westbrooke/Library/CloudStorage/OneDrive-UNC-Wilmington/SASSIE/L2/ALAMO_ALTO/'

#local directory where figures will be saved
fig_dir = '/Users/westbrooke/Library/CloudStorage/OneDrive-UNC-Wilmington/SASSIE/communication/notebooks_images/ALAMO_ALTO/'
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)


In [ ]:
##LOAD IN THE NETCDF FILES AND LOOK AT THEIR METADATA  
files = sorted(glob.glob(dir_in + '/*.nc'))
for file in files:
    ##SHOWS THE GLOBAL METEADATA AND WHICH VARIABLES ARE IN THE FILE
    ds = xr.open_dataset(file)
    #print(ds)
    
    ##SHOW THE VARIABLE ATTRIBUTES FOR ANY GIVEN VARIABLE WITHIN THE FILE. Replace "salinity" with your variable of interest:
    variable_info = ds.time
    #print(variable_info)

In [ ]:
def define_variable_attributes(var):
    if var =='salinity':
        colormap = 'viridis'
        var_label = 'Salinity'
    if var =='temperature':
        colormap = 'plasma'
        var_label = 'Water Temperature ($^{\circ}$C)'
    return colormap,var_label

In [ ]:
#PLOT THE PATHS OF THE FLOATS
def map_float_paths():

    #LOAD SHIP TRACK DATA
    ds_ship = xr.open_dataset('/Users/westbrooke/Library/CloudStorage/OneDrive-UNC-Wilmington/SASSIE/L2/TSG/SASSIE_Fall_2022_Shipboard_TSG.nc')
    ship_time = np.squeeze(ds_ship['time'])
    ship_lat = np.squeeze(ds_ship['latitude'])
    ship_lon = np.squeeze(ds_ship['longitude'])
    
    ##CREATE A MAP WITH LAND AND CITY MARKERS  
    latmin = 70
    latmax =74
    lonmin=-157
    lonmax=-144
    
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=cartopy.crs.NorthPolarStereo(central_longitude=-150))
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=cartopy.crs.PlateCarree())
    ax.coastlines(color='k')  
    ax.add_feature(cartopy.feature.LAND, facecolor = '0.50')
    ax.add_feature(cartopy.feature.RIVERS,facecolor='blue')

    
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3) 
    ax.set_title('ALTO/ALAMO Float Paths',fontsize=22,pad=1) 
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(60,75,1))
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-170,-140,2))
    gl.top_labels = False
    gl.bottom_labels = True
    gl.right_labels = False
    
    if (latmin<71.2906) & (lonmin<-156.7886):
        utqiagvik = ax.scatter(-156.7886,71.2906,s=100,transform=cartopy.crs.PlateCarree(),c='red',marker = '*',label='Utqiagvik, AK',zorder=2)
    if (latmin<70.2002) & (lonmax>-148.4597):
        deadhorse = ax.scatter(-148.4597,70.2002,s=100,c='cyan',transform=cartopy.crs.PlateCarree(),marker = '*',label='Deadhorse, AK',zorder=2)
    plt.legend(loc=2)
    
    
    ##APPLY SHIP DATA
    track = ax.plot(ship_lon, 
                 ship_lat,linewidth = 0.5,
                 c='black',
                   transform=cartopy.crs.PlateCarree(),label = 'Ship Track',zorder=1)
    plt.legend(loc=2)
    
    #LOAD AND APPLY FLOAT DATA
    for file in files: 
        ds = xr.open_dataset(file)
        float_time = np.squeeze(ds['time'])
        float_lat = np.squeeze(ds['latitude'])
        float_lon = np.squeeze(ds['longitude'])
        casts = ax.scatter(float_lon,float_lat,s = 10,
                           c = float_time,cmap = 'jet',
                           transform=cartopy.crs.PlateCarree(),zorder=2,
                          vmin=min(ship_time.values), vmax=max(float_time.values))

    cbar = fig.colorbar(casts, ax=ax, orientation="horizontal", pad=0.1)
    cbar.set_label(label='Date',size='large',weight='bold')
    cbar_tick_array=(np.linspace(ship_time.values[0].astype('int64'),float_time.values[-1].astype('int64'),5))
    cbar.set_ticks(cbar_tick_array)
    cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)
    
    ##SAVE FIGURE 
    if not os.path.exists(fig_dir+'map/'):
        os.makedirs(fig_dir+'map/')
    
    print('Saving Output Image:  '+fig_dir+'map/ALTO_ALAMO_paths.png')
    plt.savefig(fig_dir+'map/ALTO_ALAMO_paths.png',dpi='figure',format='png')
    
    


In [ ]:
map_float_paths()

In [ ]:
##USE TO PLOT THE PROFILES TAKEN BY ANY GIVEN FLOAT 
def float_profiles(float_number,var):
    
    #GET FLOAT DATA
    file = '/Users/westbrooke/Library/CloudStorage/OneDrive-UNC-Wilmington/SASSIE/L2/ALAMO_ALTO/SASSIE_Fall_2022_ALAMO_ALTO_'+str(float_number)+'.nc'
    ds_float = xr.open_dataset(file)
    float_lat = ds_float['latitude']
    float_lon = ds_float['longitude']
    float_depth = ds_float['depth']
    float_var = ds_float[var]
    
    [lat,depth] = np.meshgrid(float_lat,float_depth)
    [lon,depth] = np.meshgrid(float_lon,float_depth)
    
    
    ##CREATE 3D PLOT 
    colormap,var_label = define_variable_attributes(var)

    #To show the whole campaign area, use latmin = 70, latmax =74,lonmin=-157,lonmax=-140
    latmin = min(float_lat.values)
    latmax = max(float_lat.values)
    lonmin = min(float_lon.values)
    lonmax = max(float_lon.values)
    dmin = min(float_depth.values)
    dmax = max(float_depth.values)+0.5
    
      
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection='3d',computed_zorder=False)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False    
    
    ax.set_title('ALTO/ALAMO'+' '+var_label+' '+'Readings from Float '+str(float_number),fontsize=15,pad=None)

    ax.set_xlim(lonmin, lonmax); ax.set_ylim(latmin,latmax); ax.set_zlim(dmin,dmax);
    ax.get_xaxis().get_major_formatter().set_useOffset(False)
    xticks = list(np.linspace(lonmin,lonmax,4))
    ax.set_xticks(np.around(xticks,2))
    yticks = list(np.linspace(latmin,latmax,4))
    ax.set_yticks(np.around(yticks,2))
    zticks = list(np.linspace(dmin,dmax,4))
    ax.set_zticks(np.around(zticks,2))
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_zlabel('Depth (m)')
    ax.view_init(210,60)

        
    ##APPLY FLOAT DATA   
    a = ax.scatter(lon,lat,depth,c=(float_var.values),cmap = colormap,s=2,zorder=1)    
    fig.colorbar(a,label = var_label)
    
    ##ADD BLACK AT TOP OF EACH PROFILE FOR CLARITY
    b = ax.plot(float_lon,float_lat,linewidth=2,
                 color='black',label = 'Float Path',zorder=1)
    
    plt.legend()
    
    ## SAVE THE FIGURE
    if not os.path.exists(fig_dir+var+'/'):
        os.makedirs(fig_dir+var+'/')
    
    print('Saving Output Image:  '+fig_dir+var+'/'+'ALTO_ALAMO_'+var+'_Float_'+str(float_number)+'.png')
    plt.savefig(fig_dir+var+'/'+'ALTO_ALAMO_'+var+'_Float_'+str(float_number)+'.png',dpi='figure',format='png')
    

In [ ]:
float_profiles(11132,'temperature')
float_profiles(11133,'temperature')
float_profiles(11136,'temperature')
float_profiles(9097,'temperature')
float_profiles(9098,'temperature')
float_profiles(9101,'temperature')
float_profiles(11131,'temperature')

In [ ]:
float_profiles(11132,'salinity')
float_profiles(11133,'salinity')
float_profiles(11136,'salinity')
float_profiles(9097,'salinity')
float_profiles(9098,'salinity')
float_profiles(9101,'salinity')
float_profiles(11131,'salinity')

In [ ]:
##PLOT TEMP/SALINITY PROFILE FROM A GIVEN FLOAT CLOSEST TO A GIVEN TIME: 
def plot_float_profile_by_date(float_number,time):
    
    #GET ALL FLOAT DATA
    file = '/Users/westbrooke/Library/CloudStorage/OneDrive-UNC-Wilmington/SASSIE/L2/ALAMO_ALTO/SASSIE_Fall_2022_ALAMO_ALTO_'+str(float_number)+'.nc'
    ds_float = xr.open_dataset(file)
    float_time = ds_float['time'].astype('int64')
    
    ##FIND THE CLOSES TIME TO THE TIME SPECIFIED
    time_difference = abs((float_time.values)-np.datetime64(time,'ns').astype('int64'))
    cast_indx = np.squeeze(np.where(time_difference == min(time_difference)))
    
    #PULL OUT LAT,LON,TIME,AND DATA OF CAST
    cast_time = ds_float['time'].values[cast_indx]
    float_lat = ds_float['latitude'].values[cast_indx]
    float_lon = ds_float['longitude'].values[cast_indx]
    float_depth = ds_float['depth'].values
    float_temp = ds_float['temperature'].values[:,cast_indx]
    float_sal = ds_float['salinity'].values[:,cast_indx]
    
    
    ## PLOT THE DATA
    fig,ax1 = plt.subplots()
    ax2 = ax1.twiny()
    ax1.plot(float_temp,float_depth,linewidth=0.4,c='blue')
    ax1.invert_yaxis()
    ax1.set_xlabel('Temperature ($^{\circ}$C)',color = 'blue')
    ax1.set_ylabel('Depth (m)')
    ax2.plot(float_sal,float_depth,linewidth=0.4,c='red')
    ax2.set_xlabel('Salinity',color ='red')
    fig.suptitle('ALTO/ALAMO Float '+str(float_number)+' Salinity and Temperature Profile on '+str(pd.to_datetime(cast_time).year).zfill(2)+'/'+str(pd.to_datetime(cast_time).month).zfill(2)+'/'+str(pd.to_datetime(cast_time).day).zfill(2)+' at '+str(pd.to_datetime(cast_time).hour).zfill(2)+':'+str(pd.to_datetime(cast_time).minute).zfill(2)+':'+str(pd.to_datetime(cast_time).second).zfill(2)+'\n'+str(float_lat)+' $^{\circ}$N, '+str(float_lon)+' $^{\circ}$E',y=1.05)
    
    
    ## SAVE THE FIGURE
    if not os.path.exists(fig_dir+'profiles/'):
        os.makedirs(fig_dir+'profiles/')
    print('Saving Output Image:  '+fig_dir+'profiles/'+'float'+str(float_number)+'_profile_'+str(pd.to_datetime(cast_time).year)+str(pd.to_datetime(cast_time).month).zfill(2)+str(pd.to_datetime(cast_time).day).zfill(2)+str(pd.to_datetime(cast_time).hour).zfill(2)+str(pd.to_datetime(cast_time).minute).zfill(2)+str(pd.to_datetime(cast_time).second).zfill(2)+'.png')
    plt.savefig(fig_dir+'profiles/'+'float'+str(float_number)+'_profile_'+str(pd.to_datetime(cast_time).year)+str(pd.to_datetime(cast_time).month).zfill(2)+str(pd.to_datetime(cast_time).day).zfill(2)+str(pd.to_datetime(cast_time).hour).zfill(2)+str(pd.to_datetime(cast_time).minute).zfill(2)+str(pd.to_datetime(cast_time).second).zfill(2)+'.png',dpi='figure',format='png')



In [ ]:
plot_float_profile_by_date(9101,'2022-09-21T03:30')
